In [1]:
!pip install pyngrok fastapi

In [ ]:
import cv2
import dlib    #face point detect
import numpy as np
import json    
from datetime import datetime
import os
import math
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import base64
from io import BytesIO
import nest_asyncio
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware


HEAD_POSE_THRESHOLD = {
    "PITCH_DOWN": 60,
    "YAW_SIDEWAYS": 70,
}

YOLO_CONFIDENCE_THRESHOLD = 0.5 
YOLO_NMS_THRESHOLD = 0.4  # IoU threshold for NMS
YOLO_NMS_CONF_THRESHOLD = 0.3  # Minimum confidence for NMS


FORBIDDEN_OBJECTS = ["cell phone", "book"]

SHAPE_PREDICTOR_PATH = "/kaggle/input/dataset-test/shape_predictor_68_face_landmarks.dat"
YOLO_CFG_PATH = "/kaggle/input/dataset-test/yolov3.cfg"
YOLO_WEIGHTS_PATH = "/kaggle/input/dataset-test/yolov3.weights"
COCO_NAMES_PATH = "/kaggle/input/dataset-test/coco.names"

class SingleImageProctor:
 
    def __init__(self):
        # --- Model Initialization ---
        self.predictor = None
        self.net = None
        self.classes = None
        self.output_layers = None

        # Load dlib models
        if not os.path.exists(SHAPE_PREDICTOR_PATH):
            raise FileNotFoundError(f"Shape predictor file not found at {SHAPE_PREDICTOR_PATH}")
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor(SHAPE_PREDICTOR_PATH)

        # Load YOLO models
        if not os.path.exists(YOLO_WEIGHTS_PATH):
            raise FileNotFoundError(f"YOLO weights file not found at {YOLO_WEIGHTS_PATH}")
        if not os.path.exists(YOLO_CFG_PATH):
            raise FileNotFoundError(f"YOLO config file not found at {YOLO_CFG_PATH}")
        if not os.path.exists(COCO_NAMES_PATH):
            raise FileNotFoundError(f"COCO names file not found at {COCO_NAMES_PATH}")

        self.net = cv2.dnn.readNet(YOLO_WEIGHTS_PATH, YOLO_CFG_PATH)
        self.layer_names = self.net.getLayerNames()
        unconnected = self.net.getUnconnectedOutLayers()
        if isinstance(unconnected, np.ndarray) and unconnected.ndim > 1:
            unconnected = unconnected.flatten()
        self.output_layers = [self.layer_names[i - 1] for i in unconnected]

        with open(COCO_NAMES_PATH, "r") as f:
            self.classes = [line.strip() for line in f.readlines()]

    def _get_head_pose(self, shape, frame_dims):
      
        model_points = np.array([
            (0.0, 0.0, 0.0), 
            (0.0, -330.0, -65.0),
            (-225.0, 170.0, -135.0),  
            (225.0, 170.0, -135.0), 
            (-150.0, -150.0, -125.0),   
            (150.0, -150.0, -125.0) 
        ], dtype="double")

        image_points = np.array([
            (shape.part(30).x, shape.part(30).y),
            (shape.part(8).x, shape.part(8).y),
            (shape.part(36).x, shape.part(36).y),
            (shape.part(45).x, shape.part(45).y),
            (shape.part(48).x, shape.part(48).y),
            (shape.part(54).x, shape.part(54).y)
        ], dtype="double")

        focal_length = frame_dims[1]
        center = (frame_dims[1] / 2, frame_dims[0] / 2)
        camera_matrix = np.array(
            [[focal_length, 0, center[0]],
             [0, focal_length, center[1]],
             [0, 0, 1]], dtype="double"
        )
        dist_coeffs = np.zeros((4, 1))

        success, rotation_vector, translation_vector, inliers = cv2.solvePnPRansac(
            model_points, image_points, camera_matrix, dist_coeffs
        )

        if not success:
            return 0.0, 0.0

        theta = cv2.norm(rotation_vector, cv2.NORM_L2)
        if theta == 0:
            return 0.0, 0.0

        w = math.cos(theta / 2)
        x = math.sin(theta / 2) * rotation_vector[0][0] / theta
        y = math.sin(theta / 2) * rotation_vector[1][0] / theta
        z = math.sin(theta / 2) * rotation_vector[2][0] / theta

        ysqr = y * y
        t0 = 2.0 * (w * x + y * z)
        t1 = 1.0 - 2.0 * (x * x + ysqr)
        pitch = math.atan2(t0, t1)

        t2 = 2.0 * (w * y - z * x)
        t2 = max(min(t2, 1.0), -1.0)
        yaw = math.asin(t2)

        pitch_deg = (pitch / math.pi) * 180
        yaw_deg = (yaw / math.pi) * 180

        return pitch_deg, yaw_deg

    def _detect_forbidden_objects(self, frame):
   
        if self.net is None or self.classes is None or self.output_layers is None:
            raise ValueError("YOLO models not loaded.")

        height, width, _ = frame.shape
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)

        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > YOLO_CONFIDENCE_THRESHOLD and self.classes[class_id] in FORBIDDEN_OBJECTS:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indices = cv2.dnn.NMSBoxes(boxes, confidences, YOLO_NMS_CONF_THRESHOLD, YOLO_NMS_THRESHOLD)

        detected_objects = []
        if len(indices) > 0:
            for i in indices.flatten():
                detected_objects.append({
                    "type": "Object Detected",
                    "details": f"Forbidden object detected: {self.classes[class_ids[i]]}",
                    "confidence": confidences[i],
                    "box": boxes[i]
                })
        return detected_objects

    def analyze_image(self, frame):
    
        events = []
        frame_dims = frame.shape
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.detector(gray)

        num_faces = len(faces)
        if num_faces > 1:
            events.append({"type": "Multiple People", "details": f"{num_faces} faces detected.", "confidence": 1.0})
        elif num_faces == 0:
            events.append({"type": "Person Absent", "details": "No person detected in the image.", "confidence": 1.0})
        else:
            face = faces[0]
            shape = self.predictor(gray, face)
            pitch, yaw = self._get_head_pose(shape, frame_dims)
            if pitch > HEAD_POSE_THRESHOLD["PITCH_DOWN"]:
                excess = (pitch - HEAD_POSE_THRESHOLD["PITCH_DOWN"]) / (90 - HEAD_POSE_THRESHOLD["PITCH_DOWN"])
                conf = min(0.5 + 0.5 * excess, 1.0)
                events.append({"type": "Looking Down", "details": f"Head pitch at {pitch:.2f}°, exceeds threshold of {HEAD_POSE_THRESHOLD['PITCH_DOWN']}°.", "confidence": conf})
            if abs(yaw) > HEAD_POSE_THRESHOLD["YAW_SIDEWAYS"]:
                direction = "right" if yaw > 0 else "left"
                excess = (abs(yaw) - HEAD_POSE_THRESHOLD["YAW_SIDEWAYS"]) / (90 - HEAD_POSE_THRESHOLD["YAW_SIDEWAYS"])
                conf = min(0.5 + 0.5 * excess, 1.0)
                events.append({"type": "Looking Sideways", "details": f"Head yawed {abs(yaw):.2f}° {direction}, exceeds threshold of {HEAD_POSE_THRESHOLD['YAW_SIDEWAYS']}°.", "confidence": conf})

        try:
            object_events = self._detect_forbidden_objects(frame)
            events.extend(object_events)
        except ValueError as e:
            events.append({"type": "Model Error", "details": str(e), "confidence": 1.0})

        face_box = [faces[0].left(), faces[0].top(), faces[0].width(), faces[0].height()] if num_faces == 1 else None
        return {
            "timestamp": datetime.now().isoformat(),
            "cheating_detected": any(e["confidence"] > 0.7 for e in events),
            "events": events,
            "face_box": face_box
        }

try:
    proctor = SingleImageProctor()
except Exception as e:
    print(f"Error initializing proctor: {e}")
    raise

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], 
    allow_credentials=True,
    allow_methods=["*"], 
    allow_headers=["*"], 
)

class ImageRequest(BaseModel):
    image_base64: str

def decode_base64_image(base64_str: str):
    if base64_str.startswith("data:"):
        base64_str = base64_str.split(",", 1)[1]

    missing_padding = len(base64_str) % 4
    if missing_padding:
        base64_str += "=" * (4 - missing_padding)

    image_data = base64.b64decode(base64_str)
    nparr = np.frombuffer(image_data, np.uint8)
    image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return image

@app.post("/analyze")
async def analyze_image_endpoint(request: ImageRequest):
    try:
        image = decode_base64_image(request.image_base64)
        if image is None:
            raise ValueError("Could not decode the image from base64. Ensure it's a valid base64-encoded image without errors.")

        result = proctor.analyze_image(image)
        return result
    except base64.binascii.Error as e:
        raise HTTPException(status_code=400, detail=f"Invalid base64 encoding: {str(e)}")
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    nest_asyncio.apply()
    ngrok.set_auth_token('2s9bOqSkXKz3lX3oI41LOP5MWTi_4oJFcA4gSTB5ugts3rSY9')
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [36]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://9dcf5203a971.ngrok-free.app
